BCC Fe Exchange Model
=====================

James K. Glasbrenner

August 11, 2018

## Modules

Import standard library modules:

In [1]:
from pathlib import Path
from sys import stdout

Import external modules:

In [2]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import sympy
from scipy.constants import Boltzmann, electron_volt
from sympy import symbols
from sympy.vector import CoordSys3D
from pymatgen import units
from pymatgen.transformations.standard_transformations import SupercellTransformation
from ruamel.yaml import YAML
from sklearn.linear_model import LinearRegression

Set options, instantiate classes, and define global constants:

In [3]:
pd.set_option("display.colheader_justify", "left")
pd.set_option("display.html.border", 0)
html_table_style = {"selector": "th", "props": [("text-align", "left")]}
yaml = YAML()

kB = 1000 * Boltzmann / electron_volt

We also import functions from my own [neighbormodels](https://github.com/jkglasbrenner/datamaterials-neighbormodels) module that I developed for counting neighbors in a periodic crystal system.

In [4]:
from neighbormodels.structure import from_file
from neighbormodels.neighbors import count_neighbors
from neighbormodels.interactions import build_model

## Theory and background

### Magnetic ordering and the exchange interaction [[1](#glasbrenner2013_dissertation)]

A handful of elemental materials and a vast number of compounds exhibit phases with magnetic ordering, the spontaneous alignment of spins in the absence of an external magnetic field.
There are three general cases of ordering: ferromagnetism, antiferromagnetism, and ferrimagnetism.
In ferromagnetic ordering all spins align parallel to one another, resulting in a macroscopic magnetization.
In antiferromagnetism the spins are aligned anti-parallel to one another and the net magnetization is zero.
Unlike the ferromagnetic case, antiferromagnetic order can be achieved via different topologies of moment ordering.
In some systems, such as materials with a triangular Kagome lattice crystal structure, different antiferromagnetic orderings can be degenerate or energetically similar, giving rise to magnetic frustration.
In ferrimagnetism, the spins order in some non-ferromagnetic way, but the magnetization is finite.
For example, this can occur in compounds with two different magnetic species with different net moments, so even if the magnetic ordering is reminiscent of an antiferromagnet, the different moment magnitudes lead to a net magnetization.

The origin of magnetic ordering is the so-called exchange interaction, which refers to the effect of electrostatic interactions competing with the Pauli exclusion principle.
There are several kinds of exchange, such as *direct exchange*, *superexchange*, *indirect exchange*, *itinerant exchange*, *double exchange*, and others.
Under the right conditions, the exchange interaction leads to an energetic preference for long-range magnetic ordering, with the aforementioned different kinds of exchange often preferring one type of magnetic order over another.
Building a general exchange interaction model that is compatible with all of the different kinds of exchange mechanisms and ordering patterns remains an immense theoretical challenge.

The classical Heisenberg model is one of the models used to describe magnetic systems, and it works best in systems where the spins are localized.
There has also been success in applying this model to materials with moderate levels of itinerancy.
For the rest of this tutorial, we will focus on the details of setting up and analyzing the Heisenberg model of magnetic exchange for real material systems.

### Mean-field approximation to the Heisenberg model

The classical Heisenberg model is written as follows,

\begin{equation}
  \text{H} = \sum_{\langle{}i,j\rangle{}} J_{ij} \mathbf{m}_{i} \cdot \mathbf{m}_{j},
\end{equation}

where $i$ and $j$ are site indices, $\mathbf{m}_{i}$ is a classical vector representing the magnetic moment of site $i$, and $J_{ij}$ is the exchange constant for the interaction between sites $i$ and $j$ that parameterizes the spin-flip energy.
The range $\left\langle{}i,j\right\rangle{}$ indicates that the summation is over all possible site pairs in the lattice.

There is no known analytic solution to the classical three-dimensional Heisenberg model.
Exact numerical results are available using Monte Carlo simulations, which are straightforward to implement but can be computationally expensive.
The *mean-field approximation* (MFA) is a well-known and simple method for analyzing the Heisenberg model that has limitations, but is a reasonable starting point.
It also provides a method for computing exchange parameters using first-principles calculations by doing the following:

1.  Use density functional theory to compute the total energy of various magnetic patterns of a magnetic system.
2.  Derive mean-field expressions for the Heisenberg model mapped to the crystal structure of the magnetic system.
3.  Choose an interaction cutoff distance beyond which you assume the exchange parmeters are small enough to ignore.
    *   Model selection methods can be used to provide more rigor if the appropriate cutoff distance is not obvious.
4.  Use regression analysis to fit (train) the mean-field model on the total energy calculations to obtain the exchange parameters.

What follows is a step-by-step derivation of the mean-field model of BCC Fe, which we will use to fit to energies computed via density functional theory calculations and obtain the Fe exchange parameters.
We will also use the mean-field model to find quick estimates of the Curie temperature based on the fitted exchange parameters.

We start by defining the spin fluctuation parameter,

\begin{equation}
  \delta{}\mathbf{m}_{i} \equiv \mathbf{m}_{i} - \left\langle{} \mathbf{m}_{i} \right\rangle{},
\end{equation}

and using it to rewrite the Hamiltonian:

\begin{equation}
  \text{H} = \sum_{\langle{}i,j\rangle{}} J_{ij} \left( \left\langle{} \mathbf{m}_{i} \right\rangle{} + \delta{}\mathbf{m}_{i} \right) \cdot{} \left( \left\langle{} \mathbf{m}_{j} \right\rangle{} + \delta{}\mathbf{m}_{j} \right).
\end{equation}

We have not made any approximations yet, just transformed the variables.

Next, we expand the formula so that we can see all multiplied terms.
Let's use the [sympy](https://www.sympy.org) module to help us with the symbolic manipulations, first by declaring our variables,

In [5]:
j_ij = symbols("J_ij")
delta_m_i = symbols("𝛿m_i")
delta_m_j = symbols("𝛿m_j")
m_i_avg = symbols("<m_i>")
m_j_avg = symbols("<m_j>")
m_i = symbols("m_i")
m_j = symbols("m_j")

which allows us to to write down the formula inside the summation,

In [6]:
hamiltonian = sympy.expand(
    e = j_ij * (m_i_avg + delta_m_i) * (m_j_avg + delta_m_j)
)
print(hamiltonian)

<m_i>*<m_j>*J_ij + <m_i>*J_ij*𝛿m_j + <m_j>*J_ij*𝛿m_i + J_ij*𝛿m_i*𝛿m_j


**We now apply the mean-field approximation** by dropping spin-fluctuation terms of order $O\left(\delta{}m^{2}\right)$ and higher.

In [7]:
hamiltonian_mfa = sympy.simplify(hamiltonian.subs(delta_m_i * delta_m_j, 0))
print(hamiltonian_mfa)

J_ij*(<m_i>*<m_j> + <m_i>*𝛿m_j + <m_j>*𝛿m_i)


The mean-field Hamiltonian is,

\begin{equation}
  \text{H}_{\text{mf}} \approx \sum_{\langle{}i,j\rangle{}} J_{ij} \left( \left\langle{} \mathbf{m}_{i} \right\rangle{} \cdot{} \left\langle{} \mathbf{m}_{j} \right\rangle{} + \left\langle{} \mathbf{m}_{i} \right\rangle{} \cdot{} \delta{}\mathbf{m}_{j} + \left\langle{} \mathbf{m}_{j} \right\rangle{} \cdot{} \delta{}\mathbf{m}_{i} \right)
\end{equation}

The first term is just a constant shift of the energy, which we can remove by redefining the baseline energy for the Hamiltonian.

\begin{equation}
  \text{H}' \equiv \text{H}_{\text{mf}} + \left(\sum_{\langle{}i,j\rangle{}} J_{ij} \left\langle{} \mathbf{m}_{i} \right\rangle{} \cdot{} \left\langle{} \mathbf{m}_{j} \right\rangle{} \right)
\end{equation}

For the other two terms, we take the definition for $\delta{}\mathbf{m}_{i}$ and substitute it back into our expression,

In [8]:
print(sympy.expand(hamiltonian_mfa.subs(
    [(delta_m_i, m_i - m_i_avg),
     (delta_m_j, m_j - m_j_avg)]
)) + j_ij * m_i_avg * m_j_avg)

<m_i>*J_ij*m_j + <m_j>*J_ij*m_i


Our Hamiltonian now reads,

\begin{equation}
  \text{H}' = \sum_{\langle{}i,j\rangle{}} J_{ij} \left( \left\langle{} \mathbf{m}_{i} \right\rangle{} \cdot{} \mathbf{m}_{j} + \left\langle{} \mathbf{m}_{j} \right\rangle{} \cdot{} \mathbf{m}_{i} \right)
\end{equation}

Since the site indices $i$ and $j$ are "dummy" indices and the above dot products are commutative, we can add together the terms to further simplify the expression,

\begin{equation}
  \text{H}' = 2 \sum_{\langle{}i,j\rangle{}} J_{ij} \left\langle{} \mathbf{m}_{j} \right\rangle{} \cdot{} \mathbf{m}_{i}
\end{equation}

This form of the mean-field Hamiltonian is compact, but not the most convenient form for being able to write down models that map magnetic patterns to total energies computed using first-principles calculations.
To get our expression into a more convenient form, we transform the summation over site pairs as follows,

\begin{equation}
  \sum_{\langle{}i,j\rangle{}} \to \frac{1}{2 N_{\alpha{}}} \sum_{i} \sum_{\text{n}} \sum_{\alpha{} \leq \beta{}}
\end{equation}

The above transformation redefines how we label the site pairs, introducing the sublattices $\alpha{}$ and $\beta{}$ and the neighbor index $n$.
The factor of $\frac{1}{2}$ accounts for double-counting of pairwise bonds and the factor $\frac{1}{N_{\alpha{}}}$ normalizes the summation with respect to the total number of sublattices $N_{\alpha{}}$.
This transforms our mean-field Hamiltonian as follows,

\begin{equation}
  \text{H}' = \frac{1}{N_{\alpha}} \sum_{i} \sum_{n} \sum_{\alpha{} \leq \beta{}} z_{n}^{\alpha{}\beta{}} J_{n}^{\alpha{}\beta{}} \left\langle{} \mathbf{m}^{\beta{}} \right\rangle{} \cdot{} \mathbf{m}_{i}^{\alpha{}}
\end{equation}

For convenience, we can rewrite the summation over $\alpha{} \leq \beta{}$ into $\alpha = \beta$ and $\alpha \neq \beta$ terms,

\begin{equation}
  \text{H}' = \sum_{i} \left( \frac{1}{N_{\alpha}} \sum_{\alpha{}} \sum_{n} z_{n}^{\alpha{}} J_{n}^{\alpha{}} \left\langle{} \mathbf{m}^{\alpha{}} \right\rangle{} \cdot{} \mathbf{m}_{i}^{\alpha{}} + \frac{1}{N_{\alpha{}}} \sum_{\alpha{},\beta{}\in{}\alpha{}\neq{}\beta{}} \sum_{n} z_{n}^{\alpha{}\beta{}} J_{n}^{\alpha{}\beta{}} \left\langle{} \mathbf{m}^{\beta{}} \right\rangle{} \cdot{} \mathbf{m}_{i}^{\alpha{}} \right)
\end{equation}

### Sublattices

The sublattice concept may be familiar if you've worked with alloys, but even if you haven't, it's straightforward to understand.
For a magnetic system, a complete set of sublattices must

1.  Define a unit cell (lattice with a basis) compatible with the full periodic crystal structure
2.  Have all spins aligned in parallel within each sublattice.

### Finite temperature behavior of mean-field Heisenberg model

## Calculating exchange parameters of BCC Fe

### Crystal structure

At room temperature, [iron has the following crystal structure](#kohlhaas1967_ZAngewPhys):

<table>
  <tr><td>Structure</td><td>Body-centered cubic</td></tr>
  <tr><td>Spacegroup</td><td>Im-3m (229)</td></tr>
  <tr><td>Lattice constants</td><td>a = 2.8665 Å</td></tr>
  <tr>
    <td>Basis</td>
    <td>Fe: [0.00 0.00 0.00]</td>
  </tr>
</table>

A CIF file with these parmaeters that generates a conventional two-atom unit cell for BCC Fe is [available here](data/fe.cif).
We use this file to generate a [pymatgen](http://pymatgen.org/) `Structure` object that will be convenient for building our magnetic exchange model.

In [9]:
fe_cif_filepath = "data/fe.cif"
fe_structure_2atom = from_file(structure_file=fe_cif_filepath)

A quick printout of `fe_structure` confirms that we have the correct structure:

In [10]:
print(fe_structure_2atom)

Full Formula (Fe2)
Reduced Formula: Fe
abc   :   2.866500   2.866500   2.866500
angles:  90.000000  90.000000  90.000000
Sites (2)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Fe    0    0    0
  1  Fe    0.5  0.5  0.5


To represent the f-type, a-type, and g-type magnetic patterns, we will need a four-atom cell instead of a two-atom cell.
We transform the supercell as follows:

In [11]:
rotate_cell_45_degrees = SupercellTransformation(
    scaling_matrix=[[1, 1, 0],
                    [1, -1, 0],
                    [0, 0, 1]],
)
fe_structure_4atom = rotate_cell_45_degrees.apply_transformation(fe_structure_2atom)

As expected, this results in the required 4-atom unit cell:

In [12]:
print(fe_structure_4atom)

Full Formula (Fe4)
Reduced Formula: Fe
abc   :   4.053843   4.053843   2.866500
angles:  90.000000  90.000000  90.000000
Sites (4)
  #  SP      a    b     c
---  ----  ---  ---  ----
  0  Fe    0    0     0
  1  Fe    0.5  0.5  -0
  2  Fe    0.5  0     0.5
  3  Fe    1    0.5   0.5


### Mapping magnetic patterns onto sublattices

Defining a magnetic pattern is straightforward and is achieved by assigning a spin direction to each site in a unit cell.
We adopt the conventional method for building a classical Ising or Heisenberg model by assuming that all spin amplitudes are equal in magnitude on all sites.
This allows us to specify a magnetic pattern by assigning +1 or -1 to each site.
This means unit cells with more sites allow you to define more unique magnetic patterns, which is useful if the magnetic energy landscape is complicated such that modeling these energies requires including more terms.
For this tutorial, the BCC Fe 4-atom cell will suffice.

In [13]:
fe_magnetic_patterns = yaml.load(Path("data/fe_magnetic_patterns.yml"))

In [14]:
pd.DataFrame(fe_magnetic_patterns["4atoms"]).style \
    .set_table_styles(html_table_style) \
    .hide_index()

nm,f-type,a-type,g-type
0,1,1,1
0,1,1,-1
0,1,-1,1
0,1,-1,-1


### Counting neighbors and summing over sublattices

In [15]:
fe_neighbor_data = count_neighbors(cell_structure=fe_structure_4atom, r=3)

In [16]:
fe_exchange_model = build_model(magnetic_patterns=fe_magnetic_patterns["4atoms"], neighbor_data=fe_neighbor_data)

### Building the model matrix

In [17]:
fe_dft_energies = pd.read_csv(filepath_or_buffer="data/fe_magnetic_energies.csv") 

In [18]:
fe_model_matrix = fe_dft_energies \
    .assign(energy = lambda x:
        (x["total_energy"] -
         np.float64(x.query("pattern == 'nm'").loc[:, "total_energy"])) *
        1000 * units.Ha_to_eV / x["num_sites"]) \
    .merge(fe_exchange_model, on=["pattern"]) \
    .query("pattern != 'nm'") \
    .assign(J1 = lambda x: x["J1"]) \
    .assign(J2 = lambda x: x["J2"]) \
    .loc[:, ["pattern", "energy", "J1", "J2"]]

In [19]:
fe_model_matrix.style.set_table_styles(html_table_style).hide_index()

pattern,energy,J1,J2
f-type,-599.892,8,6
a-type,-133.324,-8,6
g-type,-375.406,0,-2


### Fitting the exchange parameters using the statsmodels module

In [20]:
smf_exchange_fit = smf.ols(data=fe_model_matrix, formula="energy ~ J1 + J2").fit()

In [21]:
smf_exchange_parameters = pd.DataFrame(smf_exchange_fit.params, columns=["statsmodels"])

### Fitting the exchange parameters using the scikit-learn module

In [22]:
lm = LinearRegression()
lm_exchange_fit = lm.fit(X=fe_model_matrix[["J1", "J2"]], y=fe_model_matrix["energy"])

In [23]:
lm_exchange_parameters = pd.DataFrame({
    "sklearn": [lm_exchange_fit.intercept_, lm_exchange_fit.coef_[0], lm_exchange_fit.coef_[1]]},
    index=["Intercept", "J1", "J2"],
)

### Parameter summary for both methods

In [24]:
smf_exchange_parameters \
    .join(lm_exchange_parameters) \
    .reset_index() \
    .rename({"index": "parameter"}, axis=1) \
    .style.set_table_styles(html_table_style) \
    .hide_index()

parameter,statsmodels,sklearn
Intercept,-373.206,-373.206
J1,-29.1605,-29.1605
J2,1.09971,1.09971


## References

1. Adapted from <a id="glasbrenner2013_dissertation"></a> [J. K. Glasbrenner, *Ab-initio and model studies of spin fluctuation effects in transport and thermodynamics of magnetic metals*, Ph.D. dissertation, University of Nebraska (2013)](http://digitalcommons.unl.edu/physicsdiss/23/).
1. <a id="kohlhaas1967_ZAngewPhys"></a> R. Kohlhaas, P. Donner, and N. Schmitz-Pranghe, Z. Angew. Phys. 23, 245 (1967)